## 학교 전력 데이터를 이용하여 net load 선형 예측해보기 - 1. 데이터 가공

### 목표: Day N의 net load data를 알 때 Day N+1의 net load를 예측하는 선형 회귀 모델을 만드는 것이 목표이다.


### 가용 데이터 종류: PV 데이터, net load 데이터


### 데이터 가공: 일단 pv 데이터는 이용하지 않고, net load 데이터들만 쭉 모아서 단순선형회귀모델을 만들어본다.


### 프로세스
1. 데이터 가공
  - 전력사용량 데이터(net load) excel에서 분석할 건물의 데이터만 쭉 뽑기
  - 비상전력계측량은 일단 고려하지 않음 ==> 고려하는 것으로 개선
  - 반복문을 사용하여 엑셀에서 column을 쭉 뽑은 후 데이터를 csv 파일로 저장
  - '-'로 표시된 누락된 데이터 존재. 이 경우 일단 Day N, N+1 관계 고려하지 않음

2. 단순선형회귀모델 찾기
 - learning rate
 - epoch
 - train:test 비율


### 보완점
다중선형회귀모델을 만들어볼 수 있을 것. PV data, 외기온도, 비상전력데이터, 주말 여부(flag 등으로 처리) 등을 고려할 수 있음.
노이즈를 추가할 수 있음.
누락된 값을 고려하여 더 정확하게 측정할 수 있음.

==> 사수님께 피드백 받은 부분:
1. 월요일부터 금요일까지의 주중 데이터만 이용해보기. calender file만 따로 만들어서 휴일에는 flag를 마크하여 data와 calender 비교해서 처리하기!
2. output이랑 y plot 해서 비교해보기! mse가 크게 나온 것은 rmse로 하면 60 정도, 전력 데이터양에 따라 달라지는 것이기 때문에 그리 정확하지 않음. 보통 평균적으로 얼만큼 전력을 쓰는지도 함께 기입하여 비교함! mse, mae, mape 등의 지표를 사용하는 것도 좋음. mape의 경우 평균치에 비해(%) 얼마나 오차가 발생했는지를 보여주기 때문에 비교하기 좋음.
3. 누락된 데이터의 경우에는 interpollation 실행. 대표적인 방법으로는 (1)전날과 다음날의 평균을 취한다. (2)전날 데이터를 그대로 가져온다. (3)전날과 가장 값이 비슷한 이전 데이터를 찾아서 그 다음날의 데이터를 취한다...


==> 보완된 부분
1. 많은 데이터를 가공하기 위해 format을 이용하여 file_path를 지정하였다.
2. Emergency data를 포함하였다.
3. 가공한 데이터를 csv 형식으로 저장해두었다.


In [ ]:
# check if pytorch is installed

!pip3 install torch torchvision                                                                                                                      

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import numpy, pandas and torch

import numpy as np
import pandas as pd
import torch
import os

In [ ]:
# connect to Google Drive

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 진짜 그냥 테스트 코드!! 열끼리 합치려고~


basic_path = '/content/drive/MyDrive/data lab internship/GIST energy data/2021 net load/undergraduate_day/학사 일보.gcf_2021-'
file_path = '{0}{1:0>2}-{2:0>2}_23-59.xls'.format(basic_path, str(1), str(1))
temp = pd.read_excel(file_path,  header=[7,8,9])
temp1 = temp["신재생에너지동"]["유효전력"]
temp2 = temp["신재생에너지동(E)"]["유효전력"]
temp = temp1+temp2
temp.columns=['21{0:0>2}{1:0>2}'.format(str(1), str(1))]
print(temp)
print(temp1)
print(temp2)

    210101
0    903.5
1    886.1
2    906.2
3    881.3
4    900.3
5    885.2
6    885.6
7    867.1
8    894.6
9    897.3
10   882.3
11   880.2
12   858.4
13   835.1
14   831.2
15   857.9
16   881.0
17   893.2
18   901.6
19   868.4
20   849.3
21   883.2
22   841.0
23   856.8
       KW
0   828.6
1   813.5
2   838.6
3   813.3
4   833.4
5   820.0
6   821.9
7   802.3
8   818.1
9   824.3
10  806.6
11  802.1
12  777.1
13  746.0
14  736.4
15  759.5
16  779.6
17  801.2
18  806.0
19  782.8
20  767.8
21  798.6
22  764.7
23  783.3
       KW
0    74.9
1    72.6
2    67.6
3    68.0
4    66.9
5    65.2
6    63.7
7    64.8
8    76.5
9    73.0
10   75.7
11   78.1
12   81.3
13   89.1
14   94.8
15   98.4
16  101.4
17   92.0
18   95.6
19   85.6
20   81.5
21   84.6
22   76.3
23   73.5


In [ ]:
# extract and combine needed colums from net load excel files into one dataframe(df)

basic_path = '/content/drive/MyDrive/data lab internship/GIST energy data/2021 net load/undergraduate_day/학사 일보.gcf_2021-'

# month list of months having only 30 days
thirty_month = [4,6,9,11]

for month in range(1,13):
  if month == 2:
    end_date = 28
  elif month in thirty_month:
    end_date = 30
  else:
    end_date = 31
  # there isn't data of end of month like 30th or 31st
  for day in range(1, end_date):
    file_path = '{0}{1:0>2}-{2:0>2}_23-59.xls'.format(basic_path, str(month), str(day))
    temp = pd.read_excel(file_path,  header=[7,8,9])
    temp1 = temp["신재생에너지동"]["유효전력"]
    temp2 = temp["신재생에너지동(E)"]["유효전력"]
    temp = temp1 + temp2

    temp.columns=['21{0:0>2}{1:0>2}'.format(str(month), str(day))]

    # avoid missing data marked with '-' by asytpe method
    try:
      temp.astype('float64')
    except ValueError:
      print('missing data! / month: %d, date: %d', month, day)
      continue

    # need to initialize dataframe df
    if month == 1 and day == 1:
      df = temp
    else:
      df = pd.concat([df, temp], axis=1)

missing data! / month: %d, date: %d 1 18
missing data! / month: %d, date: %d 1 28
missing data! / month: %d, date: %d 2 1
missing data! / month: %d, date: %d 2 21
missing data! / month: %d, date: %d 5 15
missing data! / month: %d, date: %d 6 14
missing data! / month: %d, date: %d 6 15
missing data! / month: %d, date: %d 6 16
missing data! / month: %d, date: %d 6 17
missing data! / month: %d, date: %d 6 18
missing data! / month: %d, date: %d 6 19
missing data! / month: %d, date: %d 6 20
missing data! / month: %d, date: %d 6 21
missing data! / month: %d, date: %d 10 6
missing data! / month: %d, date: %d 10 29
missing data! / month: %d, date: %d 11 6
missing data! / month: %d, date: %d 12 10


In [ ]:
df.shape

(24, 336)

In [ ]:
# make X and Y with df
# X is of Day N
# Y is of Day N+1

df = df.transpose()

X = df.iloc[0:335,:]
Y = df.iloc[1:336,:]

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
210101,903.5,886.1,906.2,881.3,900.3,885.2,885.6,867.1,894.6,897.3,...,831.2,857.9,881.0,893.2,901.6,868.4,849.3,883.2,841.0,856.8
210102,845.3,852.8,854.3,797.8,806.9,795.0,802.0,833.8,823.8,835.2,...,852.0,905.1,907.7,870.6,872.3,929.1,923.6,909.8,898.6,919.2
210103,874.1,894.2,910.5,882.1,911.1,876.2,854.1,832.9,837.5,821.3,...,852.3,868.8,900.4,935.9,884.2,923.4,929.2,889.2,893.5,893.2
210104,879.6,906.7,902.6,866.7,821.9,869.8,853.7,893.9,1063.9,1099.7,...,1103.2,1099.1,1105.6,1100.7,1043.8,1016.0,975.6,979.7,982.6,933.4
210105,925.6,936.7,882.7,894.6,864.2,859.1,853.7,891.3,957.4,1084.6,...,1159.4,1173.3,1168.5,1149.9,1105.5,976.7,978.8,989.3,965.6,968.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211226,860.1,866.2,891.8,865.9,901.3,899.8,865.2,866.0,876.0,825.8,...,822.2,797.9,818.1,832.3,870.4,833.5,876.7,879.3,876.3,859.7
211227,862.1,868.6,836.0,849.5,886.9,861.9,888.2,896.7,1085.6,1175.6,...,1127.3,1143.3,1149.3,1139.0,1111.6,1011.1,1026.3,946.0,938.4,922.8
211228,885.6,890.9,859.7,875.7,868.3,853.8,850.5,868.5,958.5,1124.9,...,1134.4,1102.5,1071.2,1091.4,997.6,928.1,933.0,862.6,902.8,896.1
211229,847.5,877.3,824.7,813.5,836.8,774.4,809.9,853.0,976.3,1021.6,...,959.4,976.4,998.9,956.7,893.3,871.3,902.9,878.8,910.0,869.2


In [ ]:
X

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
210101,903.5,886.1,906.2,881.3,900.3,885.2,885.6,867.1,894.6,897.3,...,831.2,857.9,881.0,893.2,901.6,868.4,849.3,883.2,841.0,856.8
210102,845.3,852.8,854.3,797.8,806.9,795.0,802.0,833.8,823.8,835.2,...,852.0,905.1,907.7,870.6,872.3,929.1,923.6,909.8,898.6,919.2
210103,874.1,894.2,910.5,882.1,911.1,876.2,854.1,832.9,837.5,821.3,...,852.3,868.8,900.4,935.9,884.2,923.4,929.2,889.2,893.5,893.2
210104,879.6,906.7,902.6,866.7,821.9,869.8,853.7,893.9,1063.9,1099.7,...,1103.2,1099.1,1105.6,1100.7,1043.8,1016.0,975.6,979.7,982.6,933.4
210105,925.6,936.7,882.7,894.6,864.2,859.1,853.7,891.3,957.4,1084.6,...,1159.4,1173.3,1168.5,1149.9,1105.5,976.7,978.8,989.3,965.6,968.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211225,810.9,799.7,769.8,810.6,817.2,856.7,788.2,825.8,836.8,828.8,...,882.8,870.1,952.3,907.0,909.2,890.9,870.7,928.2,863.7,813.9
211226,860.1,866.2,891.8,865.9,901.3,899.8,865.2,866.0,876.0,825.8,...,822.2,797.9,818.1,832.3,870.4,833.5,876.7,879.3,876.3,859.7
211227,862.1,868.6,836.0,849.5,886.9,861.9,888.2,896.7,1085.6,1175.6,...,1127.3,1143.3,1149.3,1139.0,1111.6,1011.1,1026.3,946.0,938.4,922.8
211228,885.6,890.9,859.7,875.7,868.3,853.8,850.5,868.5,958.5,1124.9,...,1134.4,1102.5,1071.2,1091.4,997.6,928.1,933.0,862.6,902.8,896.1


In [ ]:
Y

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
210102,845.3,852.8,854.3,797.8,806.9,795.0,802.0,833.8,823.8,835.2,...,852.0,905.1,907.7,870.6,872.3,929.1,923.6,909.8,898.6,919.2
210103,874.1,894.2,910.5,882.1,911.1,876.2,854.1,832.9,837.5,821.3,...,852.3,868.8,900.4,935.9,884.2,923.4,929.2,889.2,893.5,893.2
210104,879.6,906.7,902.6,866.7,821.9,869.8,853.7,893.9,1063.9,1099.7,...,1103.2,1099.1,1105.6,1100.7,1043.8,1016.0,975.6,979.7,982.6,933.4
210105,925.6,936.7,882.7,894.6,864.2,859.1,853.7,891.3,957.4,1084.6,...,1159.4,1173.3,1168.5,1149.9,1105.5,976.7,978.8,989.3,965.6,968.6
210106,941.6,938.5,908.3,922.8,889.1,909.2,890.2,958.8,1094.0,1174.3,...,1108.5,1095.2,1121.3,1150.4,1040.5,1043.8,1018.8,1017.6,978.0,958.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211226,860.1,866.2,891.8,865.9,901.3,899.8,865.2,866.0,876.0,825.8,...,822.2,797.9,818.1,832.3,870.4,833.5,876.7,879.3,876.3,859.7
211227,862.1,868.6,836.0,849.5,886.9,861.9,888.2,896.7,1085.6,1175.6,...,1127.3,1143.3,1149.3,1139.0,1111.6,1011.1,1026.3,946.0,938.4,922.8
211228,885.6,890.9,859.7,875.7,868.3,853.8,850.5,868.5,958.5,1124.9,...,1134.4,1102.5,1071.2,1091.4,997.6,928.1,933.0,862.6,902.8,896.1
211229,847.5,877.3,824.7,813.5,836.8,774.4,809.9,853.0,976.3,1021.6,...,959.4,976.4,998.9,956.7,893.3,871.3,902.9,878.8,910.0,869.2


In [ ]:
df.to_csv("./RISE_2021_netload.csv")